# YouTube Data API v3 Examples

In [1]:
!pip install git+https://github.com/isom-ds/social-media-api-sdk.git

  Cloning https://github.com/isom-ds/social-media-api-sdk.git to /tmp/pip-req-build-ivdhvtcu
  Running command git clone --filter=blob:none --quiet https://github.com/isom-ds/social-media-api-sdk.git /tmp/pip-req-build-ivdhvtcu
  Resolved https://github.com/isom-ds/social-media-api-sdk.git to commit 2e0d4a7f2ce1439d5f6cf85451680ad5f838a767
  Preparing metadata (setup.py) ... done
  Created wheel for uoa_isom_ds_sm_api_sdk: filename=uoa_isom_ds_sm_api_sdk-0.0.4-py3-none-any.whl size=7760 sha256=9bd57f1235e3245601bc1f3e1dc49652f36951cfb99f5c9be4faad4fd7e18132
  Stored in directory: /tmp/pip-ephem-wheel-cache-_bp4v13l/wheels/86/19/87/95af968c135dd5458ac474517bd505f4a5dab61fc2b34a356e
Successfully built uoa_isom_ds_sm_api_sdk
  Attempting uninstall: uoa_isom_ds_sm_api_sdk
    Found existing installation: uoa_isom_ds_sm_api_sdk 0.0.3
    Uninstalling uoa_isom_ds_sm_api_sdk-0.0.3:
      Successfully uninstalled uoa_isom_ds_sm_api_sdk-0.0.3


In [2]:
import uoa_isom_ds_sm_api_sdk.youtube as yt
from yt_api_key import key

## Search API

In [3]:
query = 'FTX'
search_params = {
    'key': key,
    'part': 'snippet',
    'type': 'video',
    'maxResults': 50,
    'relevanceLanguage': 'en',
    'publishedAfter': '2022-12-01' + 'T00:00:00Z',
    'publishedBefore': '2022-12-01' + 'T23:59:59Z',
    'order': 'viewCount'
}

In [4]:
search_results = await yt.fetch_search_results(query, search_params)

l_video_ids = list(set([i['id']['videoId'] for i in search_results]))
print(f"Fetched {len(l_video_ids)} Video IDs")

Fetched 14 Video IDs


## Videos API

In [5]:
video_params = {
    'key': key,
    'part': 'id,statistics,topicDetails'
}

In [6]:
# Concurrent
video_concurrent_results = await yt.fetch_videos(l_video_ids, video_params) # Also works for 1 video ID as string

print(f"Fetched data for {len(l_video_ids)} videos")
print("Video ID    | Comments")
print("------------|------------")
for k,v in video_concurrent_results.items():
    print(f"{k} | {v['statistics']['commentCount']}")

Fetched data for 14 videos
Video ID    | Comments
------------|------------
tC_orukfkGE | 27
IyoGdwVIwWw | 0
HAnFA2FAl3o | 0
8vwv23XYcrU | 13
PamZCTjqR0k | 27
CG1bpu2KqeQ | 7
7QEbjRKbrjY | 2112
0Hxf4Vf54PI | 4456
f26kRRKrlj8 | 17
Zc57tP7ODNQ | 179
RDVXI84KSY8 | 126
ipcpgND2-AY | 2
Bn0HOI9Qx0o | 52
vdv05uU1m98 | 138


## CommentThreads API

In [7]:
comments_params = {
    'key': key,
    'part': 'id,replies,snippet',
    'order': 'time'
}

# Remove videos with less than n comments
n_comments_minimum = 0
l_video_ids = [v['id'] for k,v in video_concurrent_results.items() if v['statistics']['commentCount'] >= n_comments_minimum]
print(f"{len(l_video_ids)} videos with {n_comments_minimum}+ comments")

14 videos with 0+ comments


In [8]:
# Concurrent
comments_concurrent_results = await yt.fetch_comment_threads(l_video_ids, comments_params) # Also works for 1 video ID as string

print(f"Fetched comments for {len(comments_concurrent_results)} videos")
print("Video ID    | Comments")
print("------------|------------")
for i in comments_concurrent_results.keys():
    print(f"{i} | {len(comments_concurrent_results[i])}")

Comments are disabled for video with params: {'part': 'id,replies,snippet', 'order': 'time', 'videoId': 'IyoGdwVIwWw'}
Fetched comments for 14 videos
Video ID    | Comments
------------|------------
tC_orukfkGE | 25
IyoGdwVIwWw | 0
HAnFA2FAl3o | 0
8vwv23XYcrU | 11
PamZCTjqR0k | 23
CG1bpu2KqeQ | 6
7QEbjRKbrjY | 1799
0Hxf4Vf54PI | 3390
f26kRRKrlj8 | 15
Zc57tP7ODNQ | 119
RDVXI84KSY8 | 85
ipcpgND2-AY | 1
Bn0HOI9Qx0o | 43
vdv05uU1m98 | 71


## Pipelines

In [9]:
# Search API -> Videos API -> CommentThreads API
svc_pipeline_results = await yt.search_videos_comments(query, search_params, video_params, comments_params)

print(f"Data retrieved for {len(svc_pipeline_results)} videos")
print("Video ID    | Comments")
print("------------|------------")
for k,v in svc_pipeline_results.items():
    print(f"{k} | {len(v['commentThreads'])}")

14 videos found
14 videos with 0+ comments
Comments are disabled for video with params: {'part': 'id,replies,snippet', 'order': 'time', 'videoId': 'IyoGdwVIwWw'}
Data retrieved for 14 videos
Video ID    | Comments
------------|------------
tC_orukfkGE | 25
IyoGdwVIwWw | 0
HAnFA2FAl3o | 0
8vwv23XYcrU | 11
PamZCTjqR0k | 23
CG1bpu2KqeQ | 6
7QEbjRKbrjY | 1799
0Hxf4Vf54PI | 3390
f26kRRKrlj8 | 15
Zc57tP7ODNQ | 119
RDVXI84KSY8 | 85
ipcpgND2-AY | 1
Bn0HOI9Qx0o | 43
vdv05uU1m98 | 71
